In [9]:
import sqlite3

# 1. Read the CSVs you uploaded to Colab
df_internal = pd.read_csv('/content/internal_sales.csv')
df_partner = pd.read_csv('/content/partner_reports.csv')

# 2. Create a SQL connection
conn = sqlite3.connect('audit_sandbox.db')

# 3. Push them into SQL tables
df_internal.to_sql('internal_sales', conn, if_exists='replace', index=False)
df_partner.to_sql('partner_reports', conn, if_exists='replace', index=False)

print("CSVs are now converted into SQL Tables!")

CSVs are now converted into SQL Tables!


In [3]:
# The SQL Query to find the discrepancy
query = """
SELECT
    i.transaction_id,
    i.region,
    i.actual_revenue AS internal_truth,
    p.reported_revenue AS partner_claim,
    (i.actual_revenue - p.reported_revenue) AS revenue_gap
FROM internal_sales i
JOIN partner_reports p ON i.transaction_id = p.transaction_id
WHERE i.actual_revenue <> p.reported_revenue;
"""

# Execute and store in a 'discrepancy' dataframe
df_discrepancies = pd.read_sql_query(query, conn)

print(f"Total Records Audited: 100,000")
print(f"Anomalies Found: {len(df_discrepancies)}")
df_discrepancies.head()

Total Records Audited: 100,000
Anomalies Found: 10000


,transaction_id,region,internal_truth,partner_claim,revenue_gap
0,10,South,400.77,360.69,40.08
1,23,South,595.05,535.54,59.51
2,43,South,124.80,112.32,12.48
3,66,South,257.52,231.77,25.75
4,70,South,989.04,890.14,98.90


In [4]:
# See which region is failing
print("Errors categorized by Region:")
print(df_discrepancies['region'].value_counts())

Errors categorized by Region:
region
South    10000
Name: count, dtype: int64


In [8]:
# Modular python script for future use
def automated_triage(val):
    # Using absolute value in case the gap is negative
    if abs(val) > 800:
        return " IMMEDIATE ACTION (High Loss)"
    elif abs(val) > 400:
        return " REVIEW (Medium Gap)"
    else:
        return " MONITOR (Small Variance)"


col_name = 'revenue_gap' if 'revenue_gap' in df_discrepancies.columns else 'gap'

df_discrepancies['priority_level'] = df_discrepancies[col_name].apply(automated_triage)


print("\n--- AUTOMATED REPORT GENERATED ---")
print(df_discrepancies['priority_level'].value_counts())


--- AUTOMATED REPORT GENERATED ---
priority_level
MONITOR (Small Variance)    10000
Name: count, dtype: int64
